In [1]:
#importing necessary libraries

import warnings
warnings.filterwarnings('ignore')

#Beautiful Soup will help us read the Html document
from bs4 import BeautifulSoup as soup
#allows you to send HTTP requests
from requests import get
import requests
import re
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns


In [2]:
#requesting data from url
response = requests.get("https://www.buyrentkenya.com/property-for-sale/nairobi?page=2500")
response

<Response [200]>

In [3]:
response.text[:100] #returns the first 2000 characters

'\n<!DOCTYPE html>\n<html dir="ltr" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">\n<head'

In [4]:
data = soup(response.text)
house_containers = data.find_all("div",class_= "body-icon-control-left")
title = []
for data in house_containers:
    type_ = data.find_all('a', href=True) 
    for i in type_:
        d = i['href']
        res = d.split('/')[2]
        res_1 = res.split('?')[0]
        title.append(res_1)
        
print(len(title))
# title

0


In [5]:
# %%time
# title=[]
# n_pages = 0
# for page in range(1,9): #the no. of pages to be extracted considering the pages available in the url
#     n_pages +=1
#     url = 'https://www.buyrentkenya.com/property-for-sale/nairobi?'+'&page='+str(page)
    
#     # this for gets uns into next page after every iteration.
#     r = requests.get(url) #here r is response data
#     page_html = soup(r.text)
#     house_containers = page_html.find_all("div",class_= "body-icon-control-left")
#     for data in house_containers:
#         type_ = data.find_all('a', href=True) 
#         for i in type_:
#             try:
#                 d = i['href']
#                 res = d.split('/')[2]
#                 res_1 = res.split('?')[0]
#                 title.append(res_1)
#             except:
#                 title.append('None')

# print(len(title))
# title

In [6]:
# price_per_sqft

In [7]:
#creating empty list for appending the extracted value
title = []
location = []
price = []
price_per_sqft = []
agent = []
web_link = []

In [16]:
%%time
n_pages = 0
for page in range(1,3000): #the no. of pages to be extracted considering the pages available in the url
    n_pages +=1
    url = 'https://www.buyrentkenya.com/property-for-sale/nairobi?'+'&page='+str(page)
    
    # this for gets uns into next page after every iteration.
    r = requests.get(url) #here r is response data
    page_html = soup(r.text)
    house_containers = page_html.find_all('div', class_ = 'item-body table-cell')# class containing overall data of a property
    
    ##### title
    for data in house_containers:
        type_ = data.find_all("h2", class_="property-title") 
        for i in type_:
            d = i.text
            title.append(d)
            
    ########Location 
        location_ = data.find_all("div", class_ = "property-location")
        for i in location_:
            d = i.text
            r= d.split(',',)[0]#splitting the obtained text and returning the first element of text
            location.append(r)
            
            
    ##########price
        cost = data.find_all("div", class_ = "info-row price")
        for i in cost:
            d= i.text
            if 'KES' in d:
                a = d.split()[1]
                r = re.sub(",","",a)
                price.append(r)
            else:
                e = 0
                price.append(e)
     
                
    ####### price per sqft
    
        rate_sqft = data.find_all('a', class_="item-sub-price")
        for i in rate_sqft:
            d = i.text
            if 'KES' in d:
                res = d.split('/')[0]
                r = re.sub(",","",res)
                price_per_sqft.append(r)

            else :
                e = 0
                price_per_sqft.append(e)  

print('You scraped {} pages containing {} properties.'.format(n_pages, len(title)))
# Returns the number of pages scrapped and time taken for scarpping those pages.

ConnectionError: HTTPSConnectionPool(host='www.buyrentkenya.com', port=443): Max retries exceeded with url: /property-for-sale/nairobi?&page=239 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000133413AA5C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [17]:
%%time
n_pages = 0
for page in range(1,3000): #the no. of pages to be extracted considering the pages available in the url
    n_pages +=1
    url = 'https://www.buyrentkenya.com/property-for-sale/nairobi?'+'&page='+str(page)
    
    # this for gets uns into next page after every iteration.
    r = requests.get(url) #here r is response data
    page_html = soup(r.text)
    house_containers = page_html.find_all("div",class_= "body-icon-control-left")# class containing overall data of a property
    
    ##### title
    for data in house_containers:
        type_ = data.find_all('a', href=True) 
        for i in type_:
            d = i['href']
            res = d.split('/')[2]
            res_1 = res.split('?')[0]
            agent.append(res_1)
            
print('You scraped {} pages containing {} properties.'.format(n_pages, len(agent)))

ConnectionError: HTTPSConnectionPool(host='www.buyrentkenya.com', port=443): Max retries exceeded with url: /property-for-sale/nairobi?&page=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000013341A6C128>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [18]:
df = pd.DataFrame()

In [21]:
df['title'] = title
df['location'] = location
df['price'] = price
df['agent'] = agent

ValueError: Length of values does not match length of index

In [ ]:
df['title'] = df['title'].map(lambda x: x.lstrip('\n').rstrip('\n'))

In [ ]:
df.tail()

In [ ]:
df.head()

In [ ]:
df.to_csv('all_properties_data.csv',index = False)